In [ ]:
data_dir = 'data/vortex/2020-08-16/200nm_0_25_um_step/200nm_0_25_um_stepB/'
validation_dir='data/vortex/2020-08-16/200nm_0_25_um_step/200nm_0_25_um_stepC/'
validation_cut = 0.2
fov = 0.33

In [ ]:
import math
def foveate(image, fov = fov):
    if not fov:
        return image
    centerx = image[1].shape[1]/2
    centery = image[1].shape[2]/2
    fov_pos = []
    for x,y in image[0]['XY']:
        if (math.sqrt((x-centerx)**2+(y-centery)**2) < fov * math.sqrt(centerx**2+centery**2)):
            fov_pos.append([x,y])
    image[0]['XY'] = fov_pos
    return image

In [ ]:
# Load training data
import matplotlib.pyplot as plt
import os
import numpy as np
import json
from skimage import io

print("Indexing data directory")

images = []

for filename in os.listdir(data_dir):
    if filename.endswith(".tif") and \
    os.path.isfile(os.path.join(data_dir, filename +'.json')):
        
        img = io.imread(os.path.join(data_dir, filename))
        with open(os.path.join(data_dir, filename +'.json'), 'r') as fp:
            d = json.load(fp)
        images.append(foveate([d, img]))
        
        print("Loaded " + filename)

In [ ]:
import random
import copy

validationimages = []

if validation_dir:

    print("Indexing validation data directory")

    for filename in os.listdir(validation_dir):
        if filename.endswith(".tif") and \
        os.path.isfile(os.path.join(validation_dir, filename +'.json')):

            validationimg = io.imread(os.path.join(validation_dir, filename))
            with open(os.path.join(validation_dir, filename +'.json'), 'r') as fp:
                d = json.load(fp)
            validationimages.append([d, validationimg])
            print("Loaded " + filename)
else:
    print("Doing train/test split")
    ntrain = 0
    ntest = 0
    for i in range(len(images)):
        train = []
        test = []
        shuffled = images[i][0]['XY']
        random.shuffle(shuffled)
        frac = int(math.ceil(len(shuffled)*validation_cut))
        test = shuffled[:frac]
        ntest += len(test)
        train = shuffled[frac:]
        ntrain += len(train)
        valimg = copy.deepcopy(images[i])
        images[i][0]['XY'] = train
        valimg[0]['XY'] = test
        validationimages.append(valimg)
    print("Total of {} training, {} testing spots...".format(ntrain, ntest))
        

In [ ]:
def tile_generator(images,bs, input_size, shift, shuffle = True):
    if shift<1:
        shift = int(input_size*shift)
        
    n_stacks = len(images)
    c_stack = c_spot = c_depth = 0
    
    while True:
        inputs = []
        targets = []
        while len(inputs) < bs:
            if shuffle:
                stack = random.choice(images)
                X, Y = random.choice(stack[0]['XY'])
                c = random.choice(list(stack[0]['Z'].items()))
                imgnum = int(c[0])
                Z = float(c[1])
            else:
                stack = images[c_stack]
                X, Y = stack[0]['XY'][c_spot]
                c = list(stack[0]['Z'].items())[c_depth]
                imgnum = int(c[0])
                Z = float(c[1])
                
                n_spots = len(stack[0]['XY'])
                n_depths = len(stack[0]['Z'])
                
                c_depth += 1
                
                # Move to next spot if all depths have been consumed
                if c_depth == n_depths:
                    c_depth = 0
                    c_spot += 1
                    
                    # Move to next stack if all spots have been consumed
                    if c_spot == n_spots:
                        c_spot = 0
                        c_stack += 1
                        
                        # Start a new epoch if all stacks have been consumed
                        if c_stack == n_stacks:
                            c_stack = 0
            
            xcrop, ycrop = tuple(int(p + random.uniform(-1*shift, shift)) for p in (X, Y))
            img = stack[1][imgnum][ycrop-input_size//2:ycrop+(input_size-input_size//2), \
                                  xcrop-input_size//2:xcrop+(input_size-input_size//2)]
            
            x_train = (X - (xcrop-input_size//2))/input_size
            y_train = (Y - (ycrop-input_size//2))/input_size
            z_vals = [float(v) for v in list(stack[0]['Z'].values())]
            z_train = (Z - min(z_vals)) / ((max(z_vals)-min(z_vals))/2) - 1
            try:
                # normalize image
                img = np.divide(img,np.max(img))
                img.reshape(input_size, input_size, 1)
            except:
                import pdb;pdb.set_trace()
            inputs.append(img.reshape(input_size, input_size, 1))
            targets.append(np.array([x_train, y_train, z_train]))
        yield(np.array(inputs),np.array(targets))

In [ ]:
# Instantiate generators

input_size = 64
datagen = tile_generator(images, 32, input_size, .2)
validationdatagen = tile_generator(validationimages, 32, input_size, .2)

In [ ]:
# compile model, based on https://www.pyimagesearch.com/2019/01/28/keras-regression-and-cnns/
from keras.models import Sequential
from keras.layers import *

model = Sequential()

model.add(Conv2D(16, kernel_size=(3, 3), activation='relu', input_shape=(input_size,input_size,1)))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(BatchNormalization(axis=-1))
model.add(Dropout(.5))

model.add(Dense(3, activation ='tanh'))

model.compile(loss='mse',
              optimizer='adadelta')

In [ ]:
model.summary()

In [ ]:
# train and visualize loss

history = model.fit_generator(datagen,validation_data=validationdatagen, validation_steps=100,steps_per_epoch=100, epochs=50)

In [ ]:
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
# show predictions and write results
import csv

stack = random.choice(validationimages)
n_spots = len(stack[0]['XY'])
n_depths = len(stack[0]['Z'])

testdatagen = tile_generator(validationimages, n_spots*n_depths, input_size, 0, False)
#testdatagen = tile_generator(validationimages, 50, input_size, 0, True)



batch = next(testdatagen)

print('Predicting ' + str(len(batch[0])) + " images...")

cumulative_error = 0

plot = False

%matplotlib inline

with open('res.csv', 'w', newline='') as csvfile:
    
    # Instantiate writer and write row names
    reswriter = csv.writer(csvfile)
    reswriter.writerow(['x_train', 'y_train', 'z_train', \
                        'x_pred', 'y_pred','z_pred'])
    
    for samplenum in range(len(batch[0])):

        predictions = model.predict_on_batch(batch[0])

        img = batch[0][samplenum][:,:,0]
        x_train, y_train, z = batch[1][samplenum]
        x_pred, y_pred, z_pred = predictions[samplenum]

        z_vals = [float(v) for v in list(images[0][0]['Z'].values())]
        z,z_pred = [(d + 1) * ((max(z_vals)-min(z_vals))/2) + min(z_vals) for d in [z,z_pred]]
        
        cumulative_error += abs(z-z_pred)
        
        # Write row to CSV
        reswriter.writerow([x_train, y_train, z, \
                           x_pred, y_pred, z_pred])
        
        

        print("target z:" + str(z) + '\npredicted z:' + str(z_pred))

        if plot:
            plt.figure()
            plt.imshow(img)
            plt.plot(input_size*x_train, input_size*y_train, 'bo')
            plt.plot(input_size*x_pred, input_size*y_pred, 'rX')
            plt.axis('off')
            plt.show()

avg_error = cumulative_error/len(batch[0])
print("Average error in z: {}".format(avg_error))